# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

This is the original Norvig's solution. It will be used for performance comparison. Its workflow is simple: compute all words that are at distance 1-2 from the one to correct, and then choose the one which has the biggest occurence probability

In [3]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

w = words(open('/kaggle/input/ass2base/big.txt').read())
WORDS = Counter(w)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

This is my solution. It is based on Norvig's idea, but improved to handle the context using Ngrams. The idea behind it is the following:<br>
1. The text is tokenized, only tokens that are pure alphabetic words are chosen to participate in correction
1. For each word, we correct it if it is not in our dictionary or is a short but not often word
1. We compute the candidates for the word as in Norvig's solution (the word itself and 1-2 distance words)
1. All ngram probabilities are computed. If ngrams param is 3, for example, and we want to correct the middle word in the text "a a w a a", then all ngrams will be ["w", "a w", "w a", "a a w", "a w a", "w a a"]. The probability of ngram "a a w", for example, is counts("a a w") / sum(counts("a a \*")) where \* is any word. The probabilities were computed in the preprocessing stage which is not included in the notebook, but will be described below.
1. If a word has the highest probability out of all candidates, it becomes the correction.
1. Finally, the text is postprocessed to return to the initial shape


Load the ngram dictionaries

In [12]:
import json

with open("/kaggle/input/bigrams-jsons/bigrams_key_word1.json", "r") as f:
    dct1 = json.load(f)
with open("/kaggle/input/bigrams-jsons/bigrams_key_word2.json", "r") as f:
    dct2 = json.load(f)
with open("/kaggle/input/bigrams-jsons/trigrams_key_word1_word2.json", "r") as f:
    dct3 = json.load(f)
with open("/kaggle/input/bigrams-jsons/trigrams_key_word1_word3.json", "r") as f:
    dct4 = json.load(f)
with open("/kaggle/input/bigrams-jsons/trigrams_key_word2_word3.json", "r") as f:
    dct5 = json.load(f)

In [5]:
dcts = [dct1, dct2, dct3, dct4, dct5]

Implementation itself

In [6]:
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm

class Corrector:
    """
    Context sensitive ngram spell corrector. Main method for interacting is "correct".
    """
    def __init__(self, dicts, word_set, unigram_count, ngrams=2):
        """
        The initializer of the class
        
        Params:
            dicts: the list of dictionaries to be used. For each n-gram there should be n dicts. Sorted by ascending n.
            word_set: the set of words in our language
            unigram_count: total number of words in the corpus used for unigrams (normalizing factor)
            ngrams: the maximal n for ngrams
        """
        self.dcts = dicts
        self.words = word_set
        self.first_norm = unigram_count
        self.max_word = None
        self.max_prob = 0
        self.ngrams=ngrams
        
    def edits1(word):
        """
        All edits that are one edit away from `word`. From Norvig's implementation. static method
        """
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word): 
        """
        All edits that are two edits away from `word`. From Norvig's implementation. static method
        """
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))

    def best_fit(self, text, func, penalty_scale=1):
        """
        Finds the candidate with the highest probability
        
        Params:
            text: the tokens of the part of text. Middle element is the target for correction
            func: the function to generate candidates
            penalty_scale: exponential penalty (for example, for far apart words)
        """
        # The index of the correction target
        index = len(text) // 2
        
        for e in func(text[index]):
            # The initial unigram probability
            prob = (self.words[e] + 1) / self.first_norm

            k = 0
            for i in range(2, self.ngrams+1):
                # For all ngram sizes
                
                # Generate all i-grams with the candidate in the text
                keys = []
                for j in range(i + 1):
                    ind = self.ngrams - 2 + j
                    if ind != index:
                        key = text[ind: index] + [e] + text[index + 1:ind+self.ngrams-1]
                        key = " ".join(key)
                        keys.append(key)
                
                # Search for the key in the corresponding dict
                for key in keys:
                    if key in self.dcts[k]:
                        val = self.dcts[k][key]
                        val *= len(self.words) ** (i - 2)
                        prob *= val
                    else:
                        # Avoiding zeros
                        prob *= 1e-10
                    k += 1   
            
            # Penalize if needed
            prob **= penalty_scale
            
            # Choose the best candidate
            if prob > self.max_prob:
                self.max_prob = prob
                self.max_word = e


    def fix(self, text):
        """
        Fix a single word (in the middle of the text)
        
        Params:
            text: the list of tokens. The middle one will be corrected
        Returns:
            the corrected text piece
        """
        self.max_prob = 0
        self.max_word = text[len(text) //2]

        self.best_fit(text, lambda x: [x])
        self.best_fit(text, Corrector.edits1)
        self.best_fit(text, Corrector.edits2, 1.01)

        text[1] = self.max_word
        return text

    def fix_sentence(self, text):
        """
        Fix all words in the given list of tokens.
        
        Params:
            text: list of tokens to fix
        Return:
            the fixed text
        """
        # Add paddings
        text = ['']*(self.ngrams-1) + text + ['']*(self.ngrams-1)
        
        # Correct all words one by one if needed
        for i in range(len(text) - 2):
            target = text[i+1]
            if target not in self.words or (len(target) < 4 and self.words[target] < 100):
                text[i: i+3] = self.fix(text[i: i+3])
                
        return text[self.ngrams-1: -self.ngrams+1]
    
    def correct(self, corpus, verbose=1):
        """
        The main method for interacting with the class. Is used to fix the whole corpus of the text
        
        Params:
            corpus: the text corpus
            verbose: 1 if display tqdm, 0 otherwise
        Returns:
            the fixed corpus
        """
        texts = sent_tokenize(corpus)
        corrected = []
        for text in (tqdm(texts) if verbose else texts):
            corrected.append(self.correct_one(text))
        return "\n".join(corrected)
    
    def correct_one(self, text):
        """
        Correct a single sentence. Mainlu used for preprocessing and postprocessing of the text
        """
        # Tokenize the words
        tokens = word_tokenize(text)
        
        # Take only the words that can actively participate in correction (not punctuation or numbers)
        pattern = r'^[a-zA-Z]+(\-[a-zA-Z]+)?\.?$'
        useful = [1 if re.match(pattern, t) is not None else 0 for t in tokens]
        word_toks = [t.lower() for i, t in zip(useful, tokens) if i]
        
        # Fix the sentence
        word_toks = self.fix_sentence(word_toks)
        
        # Replace the corrected words. Pretify the case of the text
        j = 0
        for i, rep in enumerate(useful):
            if rep:
                word = word_toks.pop(0)
                if tokens[i][0].isupper():
                    word = word[0].upper() + word[1:]
                if tokens[i].isupper():
                    word = word.upper()
                tokens[i] = word
        
        # Pretify punctuation
        text = " ".join(tokens)
        text = re.sub(r'\s+([^\w\s])', r'\1', text)
        return text.strip()

In [7]:
c = Corrector(dcts, WORDS, len(w), ngrams=3)

Several examples to demonstrate the work of the class

In [36]:
string = "I ws boorn on NOVEMBRE the 8th"
my = c.correct(string)
norvig = " ".join(correction(s) for s in string.split())
print(f"Original string: {string}\nCorrect: {'I was born on NOVEMBER the 8th'}\nMy correction: {my}\nNorvig's correction: {norvig}")

100%|██████████| 1/1 [00:02<00:00,  2.32s/it]

Original string: I ws boorn on NOVEMBRE the 8th
Correct:I was born on NOVEMBER the 8th
My correction: I was born on NOVEMBER the 8th
Norvig's correction: a was born on NOVEMBRE the 8th


In [37]:
string = "Chemical agents used during LrotWst at port ahgusra prixoj. Go between bridye to lprn July 5"
correct = "Chemical agents used during protest at port augusta prison. Go between bridge to open July 5"
my = c.correct(string)
norvig = " ".join(correction(s) for s in string.split())
print(f"Original string: {string}\nCorrect: {correct}\nMy correction: {my}\nNorvig's correction: {norvig}")

100%|██████████| 2/2 [00:04<00:00,  2.32s/it]


Original string: Chemical agents used during LrotWst at port ahgusra prixoj. Go between bridye to lprn July 5
Correct: Chemical agents used during protest at port augusta prison. Go between bridge to open July 5
My correction: Chemical agents used during Protest at port augusta prison.
Go between bridge to open July 5
Norvig's correction: chemical agents used during protest at port augusta prixoj. to between bridge to upon july 5


In [42]:
string = "Go fuk yourself"
correct = "Go for yourself"
my = c.correct(string)
norvig = " ".join(correction(s) for s in string.split())
print(f"Original string: {string}\nCorrect: {correct}\nMy correction: {my}\nNorvig's correction: {norvig}")

100%|██████████| 1/1 [00:00<00:00,  3.46it/s]

Original string: Go fuk yourself
Correct: Go for yourself
My correction: Go for yourself
Norvig's correction: to fur yourself


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I have made several choices in my implementation:
* Use google ngrams for bigrams and COCA for trigrams. The choice was made as the google dataset is very wide and covers a lot of real information, thus displaying the reality in a truthful manner. However, to preprocess the data I had to work with several terabytes of data (to recieve ~1GB), and I couldn't afford to work with the trigram data. So, I have chosen a more simple dataset for its availability.
* I have chosen not to correct words that contain digits or punctuation (except for . and -). This is because such misspells are extremely rare, however there are many namings containing the numbers such as "T4". Such words and punctuation were left unchanged in their places and did not participate in the correction process. The datasets were preprocessed in the corresponding manner
* I decided to add an exponential penalty to the words of distance 2. However, in my test set most of the words contained 2 misspels, and my model showed approximately the same results as Norvig's. Therefore, it was decided to lower the penalty, until it almost vanished.
* The maximal number of ngrams was chosen as 3 due to hardware limitations
* The participation of ngrams in the probability increases as the number n increases. I just multiply all of them, but there is 1 unigram, 2 bigrams, 3 trigrams, therefore each next layer has a larger impact.
* I decided to keep the original style of the text (punctuation, uppercase letters) to keep the text consistent with the original
* I decided to use Laplace smoothing on unigrams to avoid zeros, and for other bigrams I just added an insignificant constant, as using Laplace smoothing may be extremely inefficient for large n (basically starting from n >= 2)

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

Let us test the implementation. The test set is a subset of the [dataset](https://www.kaggle.com/datasets/samarthagarwal23/spelling-mistake-data-1mn), but cleaned from mistakes such as number in the middle of the word(nobody makes such mistakes, and it was decided to consider them as correct above). I used only 1000 samples and counted the accuracy of misspeled words' correction. Here are the results:

In [18]:
import pandas as pd

df = pd.read_csv("/kaggle/input/spelling-mistake-data-1mn/test.csv")
df = df[~df['augmented_text'].str.contains(r'\b\w*\d+\w*\b')]
df = df[df['augmented_text'].str.replace(" ", "").str.isalnum()]
df

,text,augmented_text
0,project looks to mulesing genetic alternative,project looks to muelsnig ngeetic alternative
1,chemical agents used during protest at port au...,chemical agents used during LrotWst at port ah...
2,business chamber seeks budget infrastructure b...,business hcmaber seeks budget infrastrcutuer b...
7,adelaide fashion festival bucks trend on recyc...,adelaide fsahoin festaivl bucks trend on recyc...
8,scott pruitt questioned on climate change in,scott pruitt quectikned on cPimzte cNZnge in
...,...,...
102479,attorney general welcomes terrorism convictions,attorney TRneral welcomes terrorism convictJohs
102480,rebels suspected as nepali politician killed,rebels usspceted as nepali politician killed
102481,lebanon lodges un protest over israeli raid,leabnno lodges un rptoest over irseali raid
102482,sydney couple accused of tabcorp fraud,ysdeny couple accused of tabcorp fraud


In [21]:
correct_mine = 0
correct_norvig = 0
total = 0

bar = tqdm(df.iloc[:1000].iterrows(), total=1000)
for row in bar:
    text_toks = row[1].text.split()
    augmented_text_toks = row[1].augmented_text.split()
    
    # Correct using my implementation
    fixed_mine = c.correct(row[1].augmented_text, verbose=0)
    fixed_mine_toks = fixed_mine.split()
    
    # Correct using Norvig's implementation
    norvig = [correction(tok) for tok in augmented_text_toks]
    
    # Calculate the words that should be corrected
    for i in range(len(text_toks)):
        if text_toks[i] != augmented_text_toks[i]:
            total += 1
            if fixed_mine_toks[i].lower() == text_toks[i]:
                correct_mine += 1
            if norvig[i] == text_toks[i]:
                correct_norvig += 1
    bar.set_postfix_str(f"Accuracy mine: {correct_mine / total: 0.4f}, accuracy Norvig's: {correct_norvig / total: 0.4f}")
                
print(f"Accuracy of my solution: {correct_mine / total: 0.4f}")
print(f"Accuracy of Norvig's solution: {correct_norvig / total: 0.4f}")

100%|██████████| 1000/1000 [37:44<00:00,  2.26s/it, Accuracy mine:  0.6150, accuracy Norvig's:  0.4021]
Accuracy of my solution:  0.6150
Accuracy of Norvig's solution:  0.4021


As we can see, my corrector scores much better results (even though the results are not perfect, probably, because of the dataset quality), so the implementation can be considered successful